# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 6 2023 11:24AM,258722,20,9235020,Else Nutrition,Released
1,Mar 6 2023 11:16AM,258721,10,Enova-11464,Emerginnova,Released
2,Mar 6 2023 11:12AM,258719,19,MSP219181,"Methapharm, Inc.",Released
3,Mar 6 2023 11:12AM,258718,10,0086481519,ISDIN Corporation,Released
4,Mar 6 2023 11:12AM,258718,10,0086481531,ISDIN Corporation,Released
5,Mar 6 2023 11:12AM,258717,19,ADV80013175,"AdvaGen Pharma, Ltd",Released
6,Mar 6 2023 11:10AM,258716,19,9234985,"Innogenix, LLC",Released
7,Mar 6 2023 11:10AM,258716,19,9234986,"Innogenix, LLC",Released
8,Mar 6 2023 11:10AM,258716,19,9234987,"Innogenix, LLC",Released
9,Mar 6 2023 11:10AM,258716,19,9234988,"Innogenix, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
15,258717,Released,1
16,258718,Released,2
17,258719,Released,1
18,258721,Released,1
19,258722,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
258717,NaN,1.0
258718,NaN,2.0
258719,NaN,1.0
258721,NaN,1.0
258722,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
258681,10.0,2.0
258688,0.0,1.0
258692,22.0,2.0
258695,5.0,24.0
258697,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
258681,10,2
258688,0,1
258692,22,2
258695,5,24
258697,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,258681,10,2
1,258688,0,1
2,258692,22,2
3,258695,5,24
4,258697,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,258681,10,2
1,258688,,1
2,258692,22,2
3,258695,5,24
4,258697,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 6 2023 11:24AM,258722,20,Else Nutrition
1,Mar 6 2023 11:16AM,258721,10,Emerginnova
2,Mar 6 2023 11:12AM,258719,19,"Methapharm, Inc."
3,Mar 6 2023 11:12AM,258718,10,ISDIN Corporation
5,Mar 6 2023 11:12AM,258717,19,"AdvaGen Pharma, Ltd"
6,Mar 6 2023 11:10AM,258716,19,"Innogenix, LLC"
29,Mar 6 2023 11:04AM,258715,10,ISDIN Corporation
30,Mar 6 2023 10:53AM,258711,10,"Citieffe, Inc."
32,Mar 6 2023 10:53AM,258709,10,"Virtue Rx, LLC"
35,Mar 6 2023 10:32AM,258705,16,Sartorius Stedim Filters Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 6 2023 11:24AM,258722,20,Else Nutrition,,1
1,Mar 6 2023 11:16AM,258721,10,Emerginnova,,1
2,Mar 6 2023 11:12AM,258719,19,"Methapharm, Inc.",,1
3,Mar 6 2023 11:12AM,258718,10,ISDIN Corporation,,2
4,Mar 6 2023 11:12AM,258717,19,"AdvaGen Pharma, Ltd",,1
5,Mar 6 2023 11:10AM,258716,19,"Innogenix, LLC",,23
6,Mar 6 2023 11:04AM,258715,10,ISDIN Corporation,,1
7,Mar 6 2023 10:53AM,258711,10,"Citieffe, Inc.",,2
8,Mar 6 2023 10:53AM,258709,10,"Virtue Rx, LLC",,3
9,Mar 6 2023 10:32AM,258705,16,Sartorius Stedim Filters Inc.,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 6 2023 11:24AM,258722,20,Else Nutrition,1,
1,Mar 6 2023 11:16AM,258721,10,Emerginnova,1,
2,Mar 6 2023 11:12AM,258719,19,"Methapharm, Inc.",1,
3,Mar 6 2023 11:12AM,258718,10,ISDIN Corporation,2,
4,Mar 6 2023 11:12AM,258717,19,"AdvaGen Pharma, Ltd",1,
5,Mar 6 2023 11:10AM,258716,19,"Innogenix, LLC",23,
6,Mar 6 2023 11:04AM,258715,10,ISDIN Corporation,1,
7,Mar 6 2023 10:53AM,258711,10,"Citieffe, Inc.",2,
8,Mar 6 2023 10:53AM,258709,10,"Virtue Rx, LLC",3,
9,Mar 6 2023 10:32AM,258705,16,Sartorius Stedim Filters Inc.,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 6 2023 11:24AM,258722,20,Else Nutrition,1,
1,Mar 6 2023 11:16AM,258721,10,Emerginnova,1,
2,Mar 6 2023 11:12AM,258719,19,"Methapharm, Inc.",1,
3,Mar 6 2023 11:12AM,258718,10,ISDIN Corporation,2,
4,Mar 6 2023 11:12AM,258717,19,"AdvaGen Pharma, Ltd",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 6 2023 11:24AM,258722,20,Else Nutrition,1.0,NaN
1,Mar 6 2023 11:16AM,258721,10,Emerginnova,1.0,NaN
2,Mar 6 2023 11:12AM,258719,19,"Methapharm, Inc.",1.0,NaN
3,Mar 6 2023 11:12AM,258718,10,ISDIN Corporation,2.0,NaN
4,Mar 6 2023 11:12AM,258717,19,"AdvaGen Pharma, Ltd",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 6 2023 11:24AM,258722,20,Else Nutrition,1.0,0.0
1,Mar 6 2023 11:16AM,258721,10,Emerginnova,1.0,0.0
2,Mar 6 2023 11:12AM,258719,19,"Methapharm, Inc.",1.0,0.0
3,Mar 6 2023 11:12AM,258718,10,ISDIN Corporation,2.0,0.0
4,Mar 6 2023 11:12AM,258717,19,"AdvaGen Pharma, Ltd",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2328342,44.0,15.0
15,258692,2.0,22.0
16,258705,1.0,0.0
19,1293551,27.0,1.0
20,258722,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2328342,44.0,15.0
1,15,258692,2.0,22.0
2,16,258705,1.0,0.0
3,19,1293551,27.0,1.0
4,20,258722,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,44.0,15.0
1,15,2.0,22.0
2,16,1.0,0.0
3,19,27.0,1.0
4,20,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,44.0
1,15,Released,2.0
2,16,Released,1.0
3,19,Released,27.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20
Status,,,,,
Executing,15.0,22.0,0.0,1.0,0.0
Released,44.0,2.0,1.0,27.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20
0,Executing,15.0,22.0,0.0,1.0,0.0
1,Released,44.0,2.0,1.0,27.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20
0,Executing,15.0,22.0,0.0,1.0,0.0
1,Released,44.0,2.0,1.0,27.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()